In [96]:
import onnx
import onnxruntime as ort
import numpy as np
import tensorflow as tf
import os
from pathlib import Path
import onnx
from onnxsim import simplify
import json 
from PIL import Image

In [97]:

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current working directory:", current_directory)


Current working directory: C:\Users\Anwender\Downloads\LectorAI_SOSE24


In [98]:

# Assuming checkpoint_path, checkpoint_dir, and colormap_path are strings representing file paths
checkpoint_path = 'dokumentenklassifikation/workspace/models/working_folder/kaggle/working/maskrcnn/MASKRCNN/checkpath_model/2024-05-24_20-57-54'
checkpoint_dir = checkpoint_path
colormap_path = checkpoint_path
checkpoint_dir_path = checkpoint_path
# Convert the paths to Path objects
checkpoint_path_obj = Path(checkpoint_path)
checkpoint_dir_obj = Path(checkpoint_dir)
colormap_path_obj = Path(colormap_path)

# Set a filename for the ONNX model
onnx_file_path = 'dokumentenklassifikation/workspace/models/working_folder/kaggle/working/maskrcnn/MASKRCNN/checkpath_model/2024-05-24_20-57-54/Bilder-maskrcnn_resnet50_fpn_v2.onnx'


In [99]:


train_sz = 1024
test_file = 'dokumentenklassifikation/MASKRCNN/test_folder/images/AD_016.jpg'  # path to test image or from frontend/backend

# Open the test file
test_img = Image.open(test_file).convert('RGB')

# Convert the resized image back to PIL format
input_img = tf.keras.preprocessing.image.img_to_array(input_img)

# Calculate the scale between the source image and the resized image
min_img_scale = min(test_img.size) / min(input_img.shape[:2])

In [100]:
# Find the colormap JSON file in the directory
colormap_path = list(checkpoint_dir_obj.glob('*colormap.json'))[0]

# Load the JSON colormap data
with open(colormap_path, 'r') as file:
    colormap_json = json.load(file)

# Convert the JSON data to a dictionary        
colormap_dict = {item['label']: item['color'] for item in colormap_json['items']}

# Extract the class names from the colormap
class_names = list(colormap_dict.keys())

# Make a copy of the colormap in integer format
int_colors = [tuple(int(c*255) for c in color) for color in colormap_dict.values()]

In [107]:
cropping_params = {
    "ad_erzieher_name":         {"left": 0.15, "bottom": 0},
    "ad_erzieher_vorname":      {"left": 0.15, "bottom": 0},
    "ad_erzieher_tel":          {"left": 0.2, "bottom": 0},
    "ad_erzieher_email":        {"left": 0.15, "bottom": 0},
    "schueler_name":            {"left": 0.15, "bottom": 0},
    "schueler_vorname":         {"left": 0.15, "bottom": 0},
    "schueler_klasse":          {"left": 0.125, "bottom": 0},
    "ad_neue_ad_str_haus_nr":   {"left": 0.275, "bottom": 0},
    "ad_neue_ad_plz":           {"left": 0.25, "bottom": 0},
    "ad_neue_ad_stadt":         {"left": 0.15, "bottom": 0},
    "ad_schueler_datum":        {"left": 0.2, "bottom": 0},
    "ag_auswahl_wahl_1":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_2":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_3":        {"left": 0.15, "bottom": 0},
    "ag_schueler_datum":        {"left": 0.3, "bottom": 0},
    "ad_schueler_unterschrift": {"left": 0.2, "bottom": 0}, 
    "ag_schueler_unterschrift": {"left": 0.2, "bottom": 0}, 
}

In [104]:

def adjust_and_crop_images(pred_labels, pred_bboxes, cropping_params, test_img):
    """
    Adjusts bounding boxes based on cropping parameters and crops the images.

    Args:
    pred_labels (list): List of predicted labels.
    pred_bboxes (list): List of predicted bounding boxes.
    cropping_params (dict): Dictionary with cropping parameters. must be adjusted in some images 
    test_img (PIL.Image): The original image to crop.

    Returns:
    list: A list of cropped images.
    """
    cropped_images = []
    
    for label, box in zip(pred_labels, pred_bboxes):
        box = box.astype(int).tolist()  # Convert tensor to list of integers
        if label in cropping_params:
            params = cropping_params[label]
            left_adjust = params.get("left", 0)
            bottom_adjust = params.get("bottom", 0)
            
            # Adjust the bounding box
            box[0] += int((box[2] - box[0]) * left_adjust)
            box[3] -= int((box[3] - box[1]) * bottom_adjust)
        
        cropped_image = test_img.crop((box[0], box[1], box[2], box[3]))
        cropped_images.append(cropped_image)
    
    return cropped_images


In [105]:
def prediction_on_image_inference(test_img, onnx_file_path, class_names, threshold=0.8):
  
    train_sz = 1024

    # Resize the test image
    input_img = tf.image.resize(test_img, [train_sz, train_sz])

     # Convert the resized image to a tensor
    input_tensor = tf.convert_to_tensor(input_img)

    # Add a batch dimension to the input tensor
    input_tensor = tf.expand_dims(input_tensor, axis=0)

    # Calculate the scale between the source image and the resized image
    min_img_scale = min(test_img.size) / min(input_img.shape[:2])    
    # Load the model and create an InferenceSession
    session = ort.InferenceSession(onnx_file_path, providers=['CPUExecutionProvider'])

    # Transpose the input tensor to match the expected format 
    input_tensor = tf.transpose(input_tensor, perm=[0, 3, 1, 2])  
    input_tensor = input_tensor.numpy().astype(np.float32) / 255.0
    
    model_output = session.run(None, {"input": input_tensor})

    # Filter the output based on the confidence threshold
    scores_mask = model_output[2] > threshold
    
    label_list = [class_names[int(idx)] for idx in model_output[1][scores_mask]]

    # Scale the predicted bounding boxes
    pred_bboxes = (model_output[0][scores_mask])*min_img_scale

    # Get the class names for the predicted label indices
    pred_labels = [class_names[int(idx)] for idx in model_output[1][scores_mask]]

    # Extract the confidence scores
    pred_scores = model_output[2]
    
    colors = [int_colors[class_names.index(i)] for i in label_list]

    # Adjust and crop the images based on the bounding boxes and cropping parameters
    cropped_images = adjust_and_crop_images(pred_labels, pred_bboxes, cropping_params, test_img)

    # Combine the bounding boxes, cropped images, labels, and confidence scores
    results = []
    for label, box, image, score in zip(pred_labels, pred_bboxes, cropped_images, pred_scores):
        results.append((label, box, image, score.item()))

    return results


In [108]:
results = prediction_on_image_inference(test_img, onnx_file_path, class_names)


In [109]:
# Display the results
for label, box, image, score in results:
    print(f"Label: {label}, Box: {box}, Confidence: {score}")
    image.show()

Label: schueler, Box: [201.02843 554.6504  767.4948  680.48376], Confidence: 0.9993707537651062
Label: ad_neue_ad, Box: [100.22949 692.0265  787.785   814.8419 ], Confidence: 0.9988994598388672
Label: ad_erzieher_name, Box: [233.00212 416.80637 752.3979  446.9802 ], Confidence: 0.9987454414367676
Label: ad_erzieher_email, Box: [224.93173 508.7705  758.86176 538.91223], Confidence: 0.9985170960426331
Label: ad_neue_ad_stadt, Box: [236.77263 780.24884 761.2298  806.95337], Confidence: 0.9983243346214294
Label: ad_erzieher, Box: [158.95529 388.83246 781.355   542.11743], Confidence: 0.9981778860092163
Label: schueler_klasse, Box: [231.70908 643.9477  757.90076 672.05695], Confidence: 0.9980339407920837
Label: ad_erzieher_vorname, Box: [202.30026 452.39795 753.15936 476.5314 ], Confidence: 0.997865617275238
Label: ad_neue_ad_str_haus_nr, Box: [115.968575 718.42426  762.1873   747.2737  ], Confidence: 0.9971975088119507
Label: ad_erzieher_tel, Box: [171.08679 480.6874  757.22626 506.32352],

In [95]:
results

[('schueler',
  array([161.90598, 553.10815, 830.1276 , 682.4923 ], dtype=float32),
  <PIL.Image.Image image mode=RGB size=669x129>,
  0.9988780617713928),
 ('ad_erzieher',
  array([109.01843, 374.5692 , 854.89325, 545.6848 ], dtype=float32),
  <PIL.Image.Image image mode=RGB size=745x171>,
  0.9987834095954895),
 ('ad_neue_ad',
  array([ 84.6607 , 687.52814, 816.2743 , 834.4951 ], dtype=float32),
  <PIL.Image.Image image mode=RGB size=732x147>,
  0.9987322688102722),
 ('ad_erzieher_name',
  array([204.4028 , 405.68066, 821.3466 , 446.98877], dtype=float32),
  <PIL.Image.Image image mode=RGB size=617x41>,
  0.9982062578201294),
 ('adress_aend',
  array([326.71072, 183.33566, 696.41327, 247.70406], dtype=float32),
  <PIL.Image.Image image mode=RGB size=370x64>,
  0.9967946410179138),
 ('ad_schueler_unterschrift',
  array([184.2755 , 908.95514, 639.8978 , 957.21454], dtype=float32),
  <PIL.Image.Image image mode=RGB size=455x49>,
  0.9966655373573303),
 ('ad_erzieher_email',
  array([214